In [12]:
from ultralytics import YOLO

import cv2
import matplotlib.pyplot as plt

In [13]:
model = YOLO("yolo11n.pt")


In [14]:
def visualize_results(results): 
    for result in results:
        img = result.orig_img  # Original image used for inference
    
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Top-left and bottom-right corner coordinates
            label = result.names[int(box.cls[0])]  # Class label
            confidence = box.conf[0]  # Confidence score
            
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)  # Blue box
            cv2.putText(img, f"{label} {confidence:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
    
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.axis("off")
        plt.show()

In [15]:
predictions = model.predict(source="./datasets/DAWN/Fog/foggy-011.jpg", show=True)

2024-11-07 23:56:22.380 Python[37970:5928923] +[IMKClient subclass]: chose IMKClient_Modern
2024-11-07 23:56:22.380 Python[37970:5928923] +[IMKInputSession subclass]: chose IMKInputSession_Modern


image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/DAWN/Fog/foggy-011.jpg: 384x640 1 car, 7 traffic lights, 122.2ms
Speed: 10.8ms preprocess, 122.2ms inference, 12.7ms postprocess per image at shape (1, 3, 384, 640)


In [16]:
import os
import numpy as np


def load_ground_truth(file_path, img_width, img_height):
    boxes = []
    with open(file_path, 'r') as file:
        for line in file:
            class_id, x_center, y_center, width, height = map(float, line.strip().split())
            x1 = int((x_center - width / 2) * img_width)
            y1 = int((y_center - height / 2) * img_height)
            x2 = int((x_center + width / 2) * img_width)
            y2 = int((y_center + height / 2) * img_height)
            boxes.append((int(class_id), x1, y1, x2, y2))
    return boxes

def calculate_iou(boxA, boxB):
    xA = max(boxA[1], boxB[1])
    yA = max(boxA[2], boxB[2])
    xB = min(boxA[3], boxB[3])
    yB = min(boxA[4], boxB[4])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    
    boxAArea = (boxA[3] - boxA[1]) * (boxA[4] - boxA[2])
    boxBArea = (boxB[3] - boxB[1]) * (boxB[4] - boxB[2])
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

def evaluate_model_on_folder(folder_path, gt_folder, model):
    all_ious = []
    for img_name in os.listdir(folder_path):
        if not img_name.endswith(".jpg"):
            continue
        img_path = os.path.join(folder_path, img_name)
        gt_path = os.path.join(gt_folder, os.path.splitext(img_name)[0] + ".txt")
        
        img = cv2.imread(img_path)
        img_height, img_width = img.shape[:2]
        
        gt_boxes = load_ground_truth(gt_path, img_width, img_height)
        results = model.predict(source=img_path)
        
        for pred in results[0].boxes:
            pred_class = int(pred.cls[0])
            pred_box = [pred_class] + list(map(int, pred.xyxy[0].tolist()))
            best_iou = 0.0
            for gt_box in gt_boxes:
                if pred_class == gt_box[0]:
                    iou = calculate_iou(pred_box, gt_box)
                    best_iou = max(best_iou, iou)
            all_ious.append(best_iou)
    
    mean_iou = np.mean(all_ious)
    return mean_iou



In [ ]:
# Run evaluation on each folder (e.g., Fog, Rain, etc.)
evaluate_model_on_folder("./datasets/DAWN/Fog", "./datasets/DAWN/Fog/Fog_YOLO_darknet")

In [17]:
import shutil
import random

In [18]:
dataset_dir = "./datasets/DAWN"
categories = ["Fog", "Rain", "Snow", "Sand"]

# Directories to store training and validation sets
train_dir = "./datasets/DAWN_split/train"
val_dir = "./datasets/DAWN_split/val"

# Create directories if they don’t exist
for folder in [train_dir, val_dir]:
    os.makedirs(folder, exist_ok=True)
    for cat in categories:
        os.makedirs(os.path.join(folder, cat), exist_ok=True)

# Split data into train and validation sets
def split_data(train_ratio=0.8):
    for category in categories:
        category_dir = os.path.join(dataset_dir, category)
        images = [f for f in os.listdir(category_dir) if f.endswith('.jpg')]
        random.shuffle(images)
        
        split_idx = int(len(images) * train_ratio)
        train_images = images[:split_idx]
        val_images = images[split_idx:]
        
        for img_name in train_images:
            shutil.copy(os.path.join(category_dir, img_name), os.path.join(train_dir, category, img_name))
            txt_name = img_name.replace(".jpg", ".txt")
            if os.path.exists(os.path.join(category_dir, f"{category}_YOLO_darknet", txt_name)):
                shutil.copy(
                    os.path.join(category_dir, f"{category}_YOLO_darknet", txt_name),
                    os.path.join(train_dir, category, txt_name)
                )
        
        for img_name in val_images:
            shutil.copy(os.path.join(category_dir, img_name), os.path.join(val_dir, category, img_name))
            txt_name = img_name.replace(".jpg", ".txt")
            if os.path.exists(os.path.join(category_dir, f"{category}_YOLO_darknet", txt_name)):
                shutil.copy(
                    os.path.join(category_dir, f"{category}_YOLO_darknet", txt_name),
                    os.path.join(val_dir, category, txt_name)
                )

# Split data
split_data()

In [19]:
model = YOLO("yolo11n.pt")

# Evaluate before training

In [20]:
print("Evaluating before training...")
initial_iou = evaluate_model_on_folder(val_dir, val_dir, model)
print("Initial Mean IoU on validation set:", initial_iou)

Evaluating before training...
Initial Mean IoU on validation set: nan


/Users/vahe/Downloads/DL_homeworks/.venv/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/vahe/Downloads/DL_homeworks/.venv/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [ ]:
# Train the model on the split data
print("Training model...")
model.train(data="./datasets/DAWN_split/data.yaml", epochs=50, imgsz=640)  # Adjust epochs as needed


In [ ]:
# Evaluate after training
print("Evaluating after training...")
final_iou = evaluate_model_on_folder(val_dir, val_dir, model)
print("Final Mean IoU on validation set:", final_iou)